# 02_03_solving_fits_and_making_sh

## 필요한 환경

이 프로젝트를 위해서는 아래의 모듈이 필요합니다.

> numpy, pandas, matplotlib, scipy, astropy, photutils, ccdproc, ysfitsutilpy, ysphotutilpy, version_information

### 모듈 버전 확인

아래 셀을 실행하면 이 노트북을 실행한 파이썬 및 관련 모듈의 버전을 확인할 수 있다.

In [1]:
import importlib, sys, subprocess
packages = "numpy, pandas, matplotlib, scipy, astropy, ccdproc, ysfitsutilpy, ysphotutilpy, version_information" # required modules
pkgs = packages.split(", ")

for pkg in pkgs :
    if not importlib.util.find_spec(pkg):
        print(f"**** module {pkg} is not installed... now start install")
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg, '-q'])
        print(f"****** module {pkg} is installed")
    else: 
        print(f"**** module {pkg} is installed")

%load_ext version_information
import time
now = time.strftime("%Y-%m-%d %H:%M:%S (%Z = GMT%z)")
print(f"This notebook was generated at {now} ")

vv = %version_information {packages}
for i, pkg in enumerate(vv.packages):
    print(f"{i} {pkg[0]:10s} {pkg[1]:s}")

**** module numpy is installed
**** module pandas is installed
**** module matplotlib is installed
**** module scipy is installed
**** module astropy is installed
**** module ccdproc is installed
**** module ysfitsutilpy is installed
**** module ysphotutilpy is installed
**** module version_information is installed
This notebook was generated at 2024-11-28 14:52:19 (KST = GMT+0900) 
0 Python     3.12.3 64bit [GCC 11.2.0]
1 IPython    8.25.0
2 OS         Linux 6.8.0 49 generic x86_64 with glibc2.39
3 numpy      1.26.4
4 pandas     2.2.2
5 matplotlib 3.8.4
6 scipy      1.13.1
7 astropy    6.1.0
8 ccdproc    2.4.2
9 ysfitsutilpy 0.2
10 ysphotutilpy 0.1.1
11 version_information 1.0.4


### import modules

In [2]:
from glob import glob
from pathlib import Path
import os
from astropy.io import fits
import astropy.units as u

from astroquery.astrometry_net import AstrometryNet

import ysfitsutilpy as yfu

import _astro_utilities
import _Python_utilities

# 환경 설정

In [3]:
#%%
#######################################################
BASEDIR = Path("/mnt/Rdata/OBS_data")  

PROJECDIR = BASEDIR / "C1-Variable"
TODODIR = PROJECDIR / "-_-_-_2016-_-_RiLA600_STX-16803_-_2bin"
TODODIR = PROJECDIR / "-_-_-_2017-01_-_RiLA600_STX-16803_-_2bin"
TODODIR = PROJECDIR / "-_-_-_2017-03_-_RiLA600_STX-16803_-_2bin"
TODODIR = PROJECDIR / "-_-_-_2017-05_-_RiLA600_STX-16803_-_2bin"
# TODODIR = PROJECDIR / "-_-_-_2017-06_-_RiLA600_STX-16803_-_2bin"
# TODODIR = PROJECDIR / "-_-_-_2021-10_-_RiLA600_STX-16803_-_2bin"
# TODODIR = PROJECDIR / "-_-_-_2022-01_-_RiLA600_STX-16803_-_2bin"

PROJECDIR = BASEDIR / "C2-Asteroid"
TODODIR = PROJECDIR / "-_-_-_2022-_-_GSON300_STF-8300M_-_1bin"
TODODIR = PROJECDIR / "-_-_-_2022-_-_RiLA600_STX-16803_-_1bin"
TODODIR = PROJECDIR / "-_-_-_2022-_-_RiLA600_STX-16803_-_2bin"
TODODIR = PROJECDIR / "-_-_-_2023-_-_GSON300_STF-8300M_-_1bin"
TODODIR = PROJECDIR / "-_-_-_2023-_-_RiLA600_STX-16803_-_1bin"
TODODIR = PROJECDIR / "-_-_-_2023-_-_RiLA600_STX-16803_-_2bin"

PROJECDIR = BASEDIR / "C3-EXO"
TODODIR = PROJECDIR / "-_-_-_2024-05_-_GSON300_STF-8300M_-_1bin"
TODODIR = PROJECDIR / "-_-_-_2024-05_-_RiLA600_STX-16803_-_1bin"
TODODIR = PROJECDIR / "-_-_-_2024-06_-_GSON300_STF-8300M_-_1bin"
# TODODIR = PROJECDIR / "-_-_-_2024-06_-_RiLA600_STX-16803_-_2bin"
# TODODIR = PROJECDIR / "-_-_-_2024-09_-_GSON300_STF-8300M_-_1bin"
# TODODIR = PROJECDIR / "-_-_-_2024-09_-_RiLA600_ASI6200MMPro_-_2bin"

# PROJECDIR = BASEDIR / "C4-Spectra"
# TODODIR = PROJECDIR / "-_-_-_2024-05_TEC140_ASI183MMPro_-_1bin"

DOINGDIRs = sorted(_Python_utilities.getFullnameListOfsubDirs(TODODIR))
print ("DOINGDIRs: ", format(DOINGDIRs))
print ("len(DOINGDIRs): ", format(len(DOINGDIRs)))

try : 
    BDFDIR = [x for x in DOINGDIRs if "CAL-BDF" in str(x)]
    print ("BDFDIR: ", format(BDFDIR))
    BDFDIR = Path(BDFDIR[0])    
except : 
    BDFDIR = TODODIR
    pass

DOINGDIRs = sorted([x for x in DOINGDIRs if "_LIGHT_" in str(x)])
# print ("DOINGDIRs: ", format(DOINGDIRs))
# print ("len(DOINGDIRs): ", format(len(DOINGDIRs)))

# filter_str = '2024-10-04'
# DOINGDIRs = [x for x in DOINGDIRs if filter_str in str(x)]
# remove = 'BIAS'
# DOINGDIRs = [x for x in DOINGDIRs if remove not in x]
# remove = 'DARK'
# DOINGDIRs = [x for x in DOINGDIRs if remove not in x]
# remove = 'FLAT'
# DOINGDIRs = [x for x in DOINGDIRs if remove not in x]
print ("DOINGDIRs: ", DOINGDIRs)
print ("len(DOINGDIRs): ", len(DOINGDIRs))
#######################################################

DOINGDIRs:  ['/mnt/Rdata/OBS_data/C3-EXO/-_-_-_2024-06_-_GSON300_STF-8300M_-_1bin/-_CAL-BDF_-_2024-05_-_GSON300_STF-8300M_-_1bin/', '/mnt/Rdata/OBS_data/C3-EXO/-_-_-_2024-06_-_GSON300_STF-8300M_-_1bin/HAT-P-23b_LIGHT_-_2024-08-03_-_GSON300_STF-8300M_-_1bin/', '/mnt/Rdata/OBS_data/C3-EXO/-_-_-_2024-06_-_GSON300_STF-8300M_-_1bin/HAT-P-23b_LIGHT_-_2024-08-31_-_GSON300_STF-8300M_-_1bin/', '/mnt/Rdata/OBS_data/C3-EXO/-_-_-_2024-06_-_GSON300_STF-8300M_-_1bin/HAT-P-37b_LIGHT_-_2024-06-28_-_GSON300_STF-8300M_-_1bin/', '/mnt/Rdata/OBS_data/C3-EXO/-_-_-_2024-06_-_GSON300_STF-8300M_-_1bin/HD189733b_LIGHT_-_2024-06-16_-_GSON300_STF-8300M_-_1bin/', '/mnt/Rdata/OBS_data/C3-EXO/-_-_-_2024-06_-_GSON300_STF-8300M_-_1bin/HD189733b_LIGHT_-_2024-06-27_-_GSON300_STF-8300M_-_1bin/', '/mnt/Rdata/OBS_data/C3-EXO/-_-_-_2024-06_-_GSON300_STF-8300M_-_1bin/Kepler-17b_LIGHT_-_2024-06-26_-_GSON300_STF-8300M_-_1bin/', '/mnt/Rdata/OBS_data/C3-EXO/-_-_-_2024-06_-_GSON300_STF-8300M_-_1bin/Qatar-10b_LIGHT_-_2024-06-02_-

In [4]:
# DOINGDIR = Path('/mnt/Rdata/OBS_data/CCD_obs_test/M13_LIGHT_-_2022-04-02_-_FS60CB_STF-8300M_-_1bin')
# print("DOINGDIR", DOINGDIR)
# SOLVINGDIR = DOINGDIR / _astro_utilities.reduced_dir
# SOLVINGDIR = DOINGDIR / _astro_utilities.reduced_nightsky_dir
# SOLVINGDIR = DOINGDIR

# summary = yfu.make_summary(SOLVINGDIR/"*.fit*")
# if summary is not None :
#     print("len(summary):", len(summary))
#     print("summary:", summary)
#     #print(summary["file"][0])  
#     df_light = summary.loc[summary["IMAGETYP"] == "LIGHT"].copy()
#     df_light = df_light.reset_index(drop=True)
#     print("df_light:\n{}".format(df_light))
# df_light


In [5]:
for DOINGDIR in DOINGDIRs :
    DOINGDIR = Path(DOINGDIR)
    print("DOINGDIR", DOINGDIR)
    SOLVINGDIR = DOINGDIR / _astro_utilities.reduced_dir
    SOLVINGDIR = DOINGDIR / _astro_utilities.reduced_nightsky_dir
    SOLVINGDIR = DOINGDIR

DOINGDIR /mnt/Rdata/OBS_data/C3-EXO/-_-_-_2024-06_-_GSON300_STF-8300M_-_1bin/HAT-P-23b_LIGHT_-_2024-08-03_-_GSON300_STF-8300M_-_1bin
DOINGDIR /mnt/Rdata/OBS_data/C3-EXO/-_-_-_2024-06_-_GSON300_STF-8300M_-_1bin/HAT-P-23b_LIGHT_-_2024-08-31_-_GSON300_STF-8300M_-_1bin
DOINGDIR /mnt/Rdata/OBS_data/C3-EXO/-_-_-_2024-06_-_GSON300_STF-8300M_-_1bin/HAT-P-37b_LIGHT_-_2024-06-28_-_GSON300_STF-8300M_-_1bin
DOINGDIR /mnt/Rdata/OBS_data/C3-EXO/-_-_-_2024-06_-_GSON300_STF-8300M_-_1bin/HD189733b_LIGHT_-_2024-06-16_-_GSON300_STF-8300M_-_1bin
DOINGDIR /mnt/Rdata/OBS_data/C3-EXO/-_-_-_2024-06_-_GSON300_STF-8300M_-_1bin/HD189733b_LIGHT_-_2024-06-27_-_GSON300_STF-8300M_-_1bin
DOINGDIR /mnt/Rdata/OBS_data/C3-EXO/-_-_-_2024-06_-_GSON300_STF-8300M_-_1bin/Kepler-17b_LIGHT_-_2024-06-26_-_GSON300_STF-8300M_-_1bin
DOINGDIR /mnt/Rdata/OBS_data/C3-EXO/-_-_-_2024-06_-_GSON300_STF-8300M_-_1bin/Qatar-10b_LIGHT_-_2024-06-02_-_GSON300_STF-8300M_-_1bin
DOINGDIR /mnt/Rdata/OBS_data/C3-EXO/-_-_-_2024-06_-_GSON300_STF-8300

In [6]:
DOINGDIR = Path(DOINGDIRs[0])
print("DOINGDIR", DOINGDIR)
SOLVINGDIR = DOINGDIR / _astro_utilities.reduced_dir
SOLVINGDIR = DOINGDIR / _astro_utilities.reduced_nightsky_dir
SOLVINGDIR = DOINGDIR

summary = yfu.make_summary(SOLVINGDIR/"*.fit*")
if summary is not None :
    print("len(summary):", len(summary))
    print("summary:", summary)
    #print(summary["file"][0])  
    df_light = summary.loc[summary["IMAGETYP"] == "LIGHT"].copy()
    df_light = df_light.reset_index(drop=True)
    print("df_light:\n{}".format(df_light))
df_light

for _, row  in df_light.iterrows():

    fpath = Path(row["file"])
    # fpath = Path(df_light["file"][1])
    print("fpath :" ,fpath)
    hdul = fits.open(fpath)

    if 'PIXSCALE' in hdul[0].header:
        PIXc = hdul[0].header['PIXSCALE']
    else : 
        PIXc = _astro_utilities.calPixScale(hdul[0].header['FOCALLEN'], 
                                            hdul[0].header['XPIXSZ'],
                                            hdul[0].header['XBINNING'])
    print("PIXc : ", PIXc)
    hdul.close()

    solved = _astro_utilities.KevinSolver(fpath, 
                                            #str(SOLVEDDIR), 
                                            # downsample = 2,
                                            # pixscale = PIXc,
                                            tryASTAP = True, 
                                            tryLOCAL = False,
                                            makeLOCALsh = True,
                                            tryASTROMETRYNET = False, 
                                            )


DOINGDIR /mnt/Rdata/OBS_data/C3-EXO/-_-_-_2024-06_-_GSON300_STF-8300M_-_1bin/HAT-P-23b_LIGHT_-_2024-08-03_-_GSON300_STF-8300M_-_1bin
All 71 keywords (guessed from /mnt/Rdata/OBS_data/C3-EXO/-_-_-_2024-06_-_GSON300_STF-8300M_-_1bin/HAT-P-23b_LIGHT_-_2024-08-03_-_GSON300_STF-8300M_-_1bin/HAT-P-23b_LIGHT_R_2024-08-03-12-01-12_120sec_GSON300_STF-8300M_29c_1bin.fit) will be loaded.


/home/guitar79/Downloads/ysfitsutilpy/ysfitsutilpy/filemgmt.py:347: UserWarning: Key CDELT1 not found for /mnt/Rdata/OBS_data/C3-EXO/-_-_-_2024-06_-_GSON300_STF-8300M_-_1bin/HAT-P-23b_LIGHT_-_2024-08-03_-_GSON300_STF-8300M_-_1bin/HAT-P-23b_LIGHT_R_2024-08-03-12-36-35_120sec_GSON300_STF-8300M_28c_1bin.fit, filling with None.
  warn(str_keyerror_fill.format(k, str(item)))
/home/guitar79/Downloads/ysfitsutilpy/ysfitsutilpy/filemgmt.py:347: UserWarning: Key CDELT2 not found for /mnt/Rdata/OBS_data/C3-EXO/-_-_-_2024-06_-_GSON300_STF-8300M_-_1bin/HAT-P-23b_LIGHT_-_2024-08-03_-_GSON300_STF-8300M_-_1bin/HAT-P-23b_LIGHT_R_2024-08-03-12-36-35_120sec_GSON300_STF-8300M_28c_1bin.fit, filling with None.
  warn(str_keyerror_fill.format(k, str(item)))
/home/guitar79/Downloads/ysfitsutilpy/ysfitsutilpy/filemgmt.py:347: UserWarning: Key CROTA1 not found for /mnt/Rdata/OBS_data/C3-EXO/-_-_-_2024-06_-_GSON300_STF-8300M_-_1bin/HAT-P-23b_LIGHT_-_2024-08-03_-_GSON300_STF-8300M_-_1bin/HAT-P-23b_LIGHT_R_2024-0

len(summary): 122
summary:                                                   file  filesize  SIMPLE  \
0    /mnt/Rdata/OBS_data/C3-EXO/-_-_-_2024-06_-_GSO...  16983360    True   
1    /mnt/Rdata/OBS_data/C3-EXO/-_-_-_2024-06_-_GSO...  16983360    True   
2    /mnt/Rdata/OBS_data/C3-EXO/-_-_-_2024-06_-_GSO...  16983360    True   
3    /mnt/Rdata/OBS_data/C3-EXO/-_-_-_2024-06_-_GSO...  17026560    True   
4    /mnt/Rdata/OBS_data/C3-EXO/-_-_-_2024-06_-_GSO...  16983360    True   
..                                                 ...       ...     ...   
117  /mnt/Rdata/OBS_data/C3-EXO/-_-_-_2024-06_-_GSO...  17026560    True   
118  /mnt/Rdata/OBS_data/C3-EXO/-_-_-_2024-06_-_GSO...  16983360    True   
119  /mnt/Rdata/OBS_data/C3-EXO/-_-_-_2024-06_-_GSO...  16983360    True   
120  /mnt/Rdata/OBS_data/C3-EXO/-_-_-_2024-06_-_GSO...  17026560    True   
121  /mnt/Rdata/OBS_data/C3-EXO/-_-_-_2024-06_-_GSO...  17026560    True   

     BITPIX  NAXIS  NAXIS1  NAXIS2  EXTEND  BZERO IMAGETYP  

In [ ]:
fpath_age = _Python_utilities.get_file_age(fpath)

In [8]:
from datetime import datetime
mtime = os.path.getmtime(fpath)

In [9]:

mtime_datetime = datetime.fromtimestamp(mtime)
now = datetime.now()
delta = now - mtime_datetime


In [10]:
delta
delta.days

86